In [52]:
import numpy as np 
import tensorflow as tf 
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Conv1D,Conv2D, Flatten, ReLU, Softmax 
from tensorflow.keras.models import Sequential
import os
MODELS_DIR = 'models/'
if not os.path.exists(MODELS_DIR):
    os.mkdir(MODELS_DIR)
MODEL_TF = MODELS_DIR + 'model'
MODEL_NO_QUANT_TFLITE = MODELS_DIR + 'model_no_quant.tflite'
MODEL_TFLITE = MODELS_DIR + 'model.tflite'
MODEL_TFLITE_MICRO = MODELS_DIR + 'model.cc'

In [53]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
X_train = X_train.reshape(-1,28,28,1)
X_test = X_test.reshape(-1,28,28,1)


In [54]:
MODEL_TF = 'models/model'

model = Sequential()

model.add(Conv2D(4, kernel_size=(3), input_shape=(28,28,1),activation='relu'))

model.add(Flatten())
model.add(Dense(10))

model.add(Softmax())

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=5)

model.summary()
model.save(MODEL_TF)

Epoch 1/5
1875/1875 [==============================] - 2s 923us/step - loss: 4.8765 - accuracy: 0.8481 - val_loss: 0.4575 - val_accuracy: 0.9352
Epoch 2/5
1875/1875 [==============================] - 2s 820us/step - loss: 0.2553 - accuracy: 0.9540 - val_loss: 0.2923 - val_accuracy: 0.9406
Epoch 3/5
1875/1875 [==============================] - 2s 827us/step - loss: 0.1357 - accuracy: 0.9656 - val_loss: 0.2084 - val_accuracy: 0.9531
Epoch 4/5
1875/1875 [==============================] - 2s 828us/step - loss: 0.0985 - accuracy: 0.9727 - val_loss: 0.1920 - val_accuracy: 0.9536
Epoch 5/5
1875/1875 [==============================] - 2s 831us/step - loss: 0.0900 - accuracy: 0.9738 - val_loss: 0.2163 - val_accuracy: 0.9470
Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 26, 26, 4)         40        
______________________________________________________

/Users/anubhav/miniforge3/envs/tf_macos/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
/Users/anubhav/miniforge3/envs/tf_macos/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer.py:1397: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`layer.updates` will be removed in a future version. '



FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.
INFO:tensorflow:Assets written to: models/model/assets


In [55]:
model.predict(X_test)

array([[2.6167428e-09, 1.2372639e-14, 1.6009518e-07, ..., 9.7390670e-01,
        4.2211128e-09, 2.0113382e-07],
       [6.9597017e-10, 7.1916190e-10, 1.0000000e+00, ..., 6.1048779e-18,
        7.4545783e-09, 8.9033158e-16],
       [1.8634055e-05, 9.9802434e-01, 6.7517493e-04, ..., 1.9321169e-06,
        1.1350051e-03, 1.9921413e-06],
       ...,
       [1.9272928e-09, 3.2649456e-08, 2.0605258e-09, ..., 1.2136607e-02,
        2.2476805e-04, 1.1804704e-04],
       [4.1797178e-12, 3.2030686e-16, 7.1189764e-12, ..., 5.0552092e-13,
        1.2813935e-04, 1.8515460e-09],
       [3.0831888e-09, 8.5864111e-15, 1.5822263e-09, ..., 7.2210596e-14,
        7.4529924e-11, 9.5907455e-16]], dtype=float32)

In [56]:
MODEL_TF = 'models/model'
MODEL_NO_QUANT_TFLITE = 'models/model_no_quant.tflite'
MODEL_TFLITE = 'models/model.tflite'
MODEL_TFLITE_MICRO = 'models/model.cc'

   
#model without quantization    
converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_TF)
model_no_quant_tflite = converter.convert()
# Save the model to disk
open(MODEL_NO_QUANT_TFLITE, "wb").write(model_no_quant_tflite)


def representative_dataset():
    for i in range(500):
        yield([(X_train[i].reshape(-1,784,1)).astype(np.float32)])

# Set the optimization flag.
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
# Provide a representative dataset to ensure we quantize correctly.
converter.representative_dataset = representative_dataset
model_tflite = converter.convert()
# Save the model to disk
open(MODEL_TFLITE, "wb").write(model_tflite)

y_test_pred_tf = model.predict(X_test)


ConverterError: <unknown>:0: error: loc(callsite(callsite("sequential_16/conv2d_11/Conv2D@__inference__wrapped_model_196976" at "StatefulPartitionedCall@__inference_signature_wrapper_197159") at "StatefulPartitionedCall")): 'tf.MLCConv2D' op is neither a custom op nor a flex op
<unknown>:0: note: loc("StatefulPartitionedCall"): called from
<unknown>:0: error: loc(callsite(callsite("sequential_16/dense_13/MLCMatMul@__inference__wrapped_model_196976" at "StatefulPartitionedCall@__inference_signature_wrapper_197159") at "StatefulPartitionedCall")): 'tf.MLCMatMul' op is neither a custom op nor a flex op
<unknown>:0: note: loc("StatefulPartitionedCall"): called from
<unknown>:0: error: failed while converting: 'main': Ops that need custom implementation (enabled via setting the -emit-custom-ops flag):
	tf.MLCConv2D {T = f32, data_format = "NHWC", device = "", dilations = [1, 1, 1, 1], explicit_paddings = [], num_args = 0 : i64, padding = "VALID", strides = [1, 1, 1, 1], transpose = false, u

In [7]:
def predict_tflite(tflite_model, x_test):
  # Prepare the test data
  x_test_ = x_test.copy()
  x_test_ = x_test_.reshape((-1,784,1))
  x_test_ = x_test_.astype(np.float32)

  # Initialize the TFLite interpreter
  interpreter = tf.lite.Interpreter(model_content=tflite_model)
  interpreter.allocate_tensors()

  input_details = interpreter.get_input_details()[0]
  output_details = interpreter.get_output_details()[0]

  # If required, quantize the input layer (from float to integer)
  input_scale, input_zero_point = input_details["quantization"]
  if (input_scale, input_zero_point) != (0.0, 0):
    x_test_ = x_test_ / input_scale + input_zero_point
    x_test_ = x_test_.astype(input_details["dtype"])
  
  # Invoke the interpreter
  y_pred = np.empty((x_test_.shape[0],10), dtype=output_details["dtype"])
  for i in range(len(x_test_)):
    interpreter.set_tensor(input_details["index"], [x_test_[i]])
    interpreter.invoke()
    y_pred[i] = interpreter.get_tensor(output_details["index"])[0]
  
  # If required, dequantized the output layer (from integer to float)
  output_scale, output_zero_point = output_details["quantization"]
  if (output_scale, output_zero_point) != (0.0, 0):
    y_pred = y_pred.astype(np.float32)
    y_pred = (y_pred - output_zero_point) * output_scale

  return y_pred

def evaluate_tflite(tflite_model, x_test, y_true):
  global model
  y_pred = predict_tflite(tflite_model, x_test)
  cce = tf.keras.losses.CategoricalCrossentropy()  
  loss = cce(y_true, y_pred).numpy()
  return loss


In [20]:
y_test_pred_tf = model.predict(X_test)
y_test_pred_no_quant_tflite = predict_tflite(model_no_quant_tflite, X_test)
y_test_pred_tflite = predict_tflite(model_tflite, X_test)

In [21]:
loss_tf, _ = model.evaluate(X_test, y_test, verbose=0)
loss_no_quant_tflite = evaluate_tflite(model_no_quant_tflite, X_test, y_test)
loss_tflite = evaluate_tflite(model_tflite, X_test, y_test)

In [28]:
import pandas as pd
df = pd.DataFrame.from_records(
    [["TensorFlow", loss_tf],
     ["TensorFlow Lite", loss_no_quant_tflite],
     ["TensorFlow Lite Quantized", loss_tflite]],
     columns = ["Model", "Loss/MSE"], index="Model").round(4)
df

NameError: name 'loss_tf' is not defined

In [48]:
size_tf = os.path.getsize(MODEL_TF)
size_no_quant_tflite = os.path.getsize(MODEL_NO_QUANT_TFLITE)
size_tflite = os.path.getsize(MODEL_TFLITE)
pd.DataFrame.from_records(
    [["TensorFlow", f"{size_tf} bytes", ""],
     ["TensorFlow Lite", f"{size_no_quant_tflite} bytes ", f"(reduced by {size_tf - size_no_quant_tflite} bytes)"],
     ["TensorFlow Lite Quantized", f"{size_tflite} bytes", f"(reduced by {size_no_quant_tflite - size_tflite} bytes)"]],
     columns = ["Model", "Size", ""], index="Model")

,Size,
Model,,
TensorFlow,224 bytes,
TensorFlow Lite,56828 bytes,(reduced by -56604 bytes)
TensorFlow Lite Quantized,16552 bytes,(reduced by 40276 bytes)


In [51]:
# Install xxd if it is not available
!apt-get update && apt-get -qq install xxd

# Convert to a C source file, i.e, a TensorFlow Lite for Microcontrollers model
!xxd -i {'models/model.tflite'} > {'models/model.cc'}
# Update variable names
REPLACE_TEXT = MODEL_TFLITE.replace('/', '_').replace('.', '_')
!sed -i 's/'{REPLACE_TEXT}'/g_model/g' {MODEL_TFLITE_MICRO}

zsh:1: command not found: apt-get
sed: 1: "models/model.cc": invalid command code m


In [50]:
!cat {MODEL_TFLITE_MICRO}

unsigned char models_model_tflite[] = {
  0x1c, 0x00, 0x00, 0x00, 0x54, 0x46, 0x4c, 0x33, 0x14, 0x00, 0x20, 0x00,
  0x1c, 0x00, 0x18, 0x00, 0x14, 0x00, 0x10, 0x00, 0x0c, 0x00, 0x00, 0x00,
  0x08, 0x00, 0x04, 0x00, 0x14, 0x00, 0x00, 0x00, 0x1c, 0x00, 0x00, 0x00,
  0xa4, 0x00, 0x00, 0x00, 0xd4, 0x00, 0x00, 0x00, 0xcc, 0x36, 0x00, 0x00,
  0xdc, 0x36, 0x00, 0x00, 0xdc, 0x3f, 0x00, 0x00, 0x03, 0x00, 0x00, 0x00,
  0x01, 0x00, 0x00, 0x00, 0x10, 0x00, 0x00, 0x00, 0x0c, 0x00, 0x14, 0x00,
  0x10, 0x00, 0x0c, 0x00, 0x08, 0x00, 0x04, 0x00, 0x0c, 0x00, 0x00, 0x00,
  0x10, 0x00, 0x00, 0x00, 0x14, 0x00, 0x00, 0x00, 0x24, 0x00, 0x00, 0x00,
  0x44, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00,
  0x0f, 0x00, 0x00, 0x00, 0x73, 0x65, 0x72, 0x76, 0x69, 0x6e, 0x67, 0x5f,
  0x64, 0x65, 0x66, 0x61, 0x75, 0x6c, 0x74, 0x00, 0x01, 0x00, 0x00, 0x00,
  0x04, 0x00, 0x00, 0x00, 0xb8, 0xff, 0xff, 0xff, 0x0e, 0x00, 0x00, 0x00,
  0x04, 0x00, 0x00, 0x00, 0x09, 0x00, 0x00, 0x00, 0x73, 0x6f, 0x66, 0x74